# Jigsaw generation #1

In [1]:
import numpy as np

from jigsaw import *
from jigsaw.default import *
from jigsaw.svg import *

## Simple 3x3

In [2]:
# Choose only centered edge types, two sizes
edge_types = np.array([
    CENTERED_SMALL_MALE,
    CENTERED_MEDIUM_MALE,
])

# Automatically expand types to match combinations
edge_types = expand_edge_types(edge_types, opposite, flip)

# Show actual types
edge_names[edge_types]

array(['CENTERED_MEDIUM_MALE', 'CENTERED_MEDIUM_FEMALE',
       'CENTERED_SMALL_MALE', 'CENTERED_SMALL_FEMALE'], dtype='<U22')

In [3]:
# Generate grid
# TODO unique solution loop
H, W = 1, 2
horizontal_edges, vertical_edges = build_random_grid(H, W, edge_types)

In [4]:
display_grid(horizontal_edges, vertical_edges)

In [5]:
pieces = grid_to_pieces(horizontal_edges, vertical_edges, opposite)
for h, v in iterate_solutions(H, W, pieces, flip, opposite, constraints="border"):
    display_grid(h, v)

In [5]:
display_grid(*flip_grid(horizontal_edges, vertical_edges, opposite, flip))

In [6]:
pieces = np.array([[[CENTERED_MEDIUM_MALE,CENTERED_MEDIUM_FEMALE, LEFT_SMALL_FEMALE,LEFT_SMALL_FEMALE]], [[TWISTED_SMALL_FEMALE, RIGHT_SMALL_MALE, CENTERED_SMALL_MALE, CENTERED_SMALL_FEMALE]]])
horizontal_edges, vertical_edges = pieces_to_grid(pieces, opposite)
display_grid(horizontal_edges, vertical_edges)

In [7]:
def flip_grid(horizontal_edges, vertical_edges, opposite, flip):
    h = opposite[flip[vertical_edges]].T
    v = opposite[flip[horizontal_edges]].T
    return h, v

In [8]:
display_grid(*flip_grid(horizontal_edges, vertical_edges, opposite, flip))

In [9]:
display_grid(horizontal_edges, vertical_edges)

In [10]:
horizontal_edges.shape

(2, 2)

In [11]:
edge_names[horizontal_edges]

array([['LEFT_SMALL_FEMALE', 'CENTERED_MEDIUM_FEMALE'],
       ['CENTERED_SMALL_MALE', 'TWISTED_SMALL_FEMALE']], dtype='<U22')

In [12]:
edge_names[vertical_edges]

array([['CENTERED_MEDIUM_MALE'],
       ['LEFT_SMALL_FEMALE'],
       ['CENTERED_SMALL_FEMALE']], dtype='<U22')

In [13]:
display_grid(*rotate_grid_cw(horizontal_edges, vertical_edges, opposite))

In [22]:
horizontal_edges, vertical_edges = build_random_grid(3, 3, edge_types)

In [23]:
pieces = grid_to_pieces(horizontal_edges, vertical_edges, opposite)
pieces, C, N, count, canonical, accept, horizontal, vertical = prepare_solver_arguments(3, 3, pieces, opposite, flip, "border")

In [8]:
for stack in iterate_solver(C, N, 3, 3, count, canonical, accept, horizontal, vertical):
    break

## Pack as a single grid

In [ ]:
# TODO